## **Computer vision project: Deggendorf Waste Sorting Assistant**

### **Overview**
The Deggendorf Waste Sorting Assistant is a Computer Vision-based tool designed to help residents and international students correctly identify waste bins. The project leverages image classification to determine the category of a given waste bin based on its visual characteristics. Users can take a picture of an unlabeled bin, and the model will classify it while providing information on the appropriate waste materials for disposal.

### **Project Goals**
- Develop an image classification model capable of identifying waste bins in Deggendorf.
- Provide users with clear guidance on proper waste disposal based on bin classification.
- Document all processes in a Jupyter Notebook, covering dataset creation, model training, evaluation, and deployment.


### 1. Mount Google Drive & Interactive Labeling Utility

This section sets up everything you need to label images **in-Colab**:

1. Installs required packages  
2. Mounts your Drive  
3. Enables Colab’s custom widget manager for `ipywidgets`  
4. Defines constants, logging, and a CSV to track labels  
5. Provides an interactive widget UI to:
   - Scan `/MyDrive/cv_garbage` for unlabeled images  
   - Display one image at a time  
   - Pick a label from a fixed list  
   - Copy the image into `/MyDrive/cv_garbage/labled` with a standardized name  
   - Record `original_filename`, `new_filename`, `label`, and `timestamp` in `labels.csv`  

---

In [ ]:
# 1.0 Install dependencies
%pip install --quiet --upgrade pandas pillow ipywidgets

In [ ]:
# 1.1 Mount Drive & enable widgets
from google.colab import drive, output
drive.mount('/content/drive', force_remount=True)
output.enable_custom_widget_manager()


In [ ]:
# 1.2 Imports, Paths & Logging
import os, shutil, logging, pandas as pd
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image as PILImage
import ipywidgets as widgets

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s: %(message)s")

BASE_DIR    = '/content/drive/MyDrive/cv_garbage'
LABELED_DIR = os.path.join(BASE_DIR, 'labled')
CSV_PATH    = os.path.join(LABELED_DIR, 'labels.csv')
os.makedirs(LABELED_DIR, exist_ok=True)

LABELS = ["Restmüll", "Biomüll", "Papier", "Gelber_Sack", "Glas"]


In [ ]:
# 1.2 Imports, Paths & Logging
import os, shutil, logging, pandas as pd
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image as PILImage
import ipywidgets as widgets

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s: %(message)s")

BASE_DIR    = '/content/drive/MyDrive/cv_garbage'
LABELED_DIR = os.path.join(BASE_DIR, 'labled')
CSV_PATH    = os.path.join(LABELED_DIR, 'labels.csv')
os.makedirs(LABELED_DIR, exist_ok=True)

LABELS = ["Restmüll", "Biomüll", "Papier", "Gelber_Sack", "Glas"]


In [ ]:
# 1.3 Labeler Class Definition
class ColabImageLabeler:
    def __init__(self, src_dir, dst_dir, csv_path, labels):
        self.src, self.dst, self.csv_path, self.labels = src_dir, dst_dir, csv_path, labels
        self.df     = self._load_or_init_csv()
        self.images = self._get_unlabeled_images()
        self.i      = 0

        self.out     = widgets.Output()
        self.dropdown= widgets.Dropdown(options=self.labels, description='Label:')
        self.button  = widgets.Button(description='Next ▶️')
        self.status  = widgets.Label()
        self.button.on_click(self._on_next)

    def _load_or_init_csv(self):
        if os.path.exists(self.csv_path):
            df = pd.read_csv(self.csv_path)
            logging.info(f"Loaded {len(df)} entries.")
        else:
            df = pd.DataFrame(columns=['original_filename','new_filename','label','timestamp'])
            logging.info("Initialized new CSV.")
        return df

    def _get_unlabeled_images(self):
        exts = {'.jpg','.jpeg','.png','.bmp','.gif'}
        all_imgs = sorted(f for f in os.listdir(self.src)
                          if os.path.splitext(f.lower())[1] in exts)
        labeled = set(self.df['original_filename'])
        unlabeled = [f for f in all_imgs if f not in labeled]
        logging.info(f"{len(unlabeled)} unlabeled images found.")
        return unlabeled

    def _show_image(self, path):
        with self.out:
            clear_output(wait=True)
            display(PILImage.open(path).resize((480,360)))

    def _on_next(self, _):
        if self.i >= len(self.images): return
        orig = self.images[self.i]
        src  = os.path.join(self.src, orig)
        lbl  = self.dropdown.value.replace(' ', '_')
        ext  = os.path.splitext(orig)[1]
        new  = f"{lbl}_{self.i+1:04d}{ext}"
        dst  = os.path.join(self.dst, new)
        timestamp = datetime.utcnow().isoformat()

        try:
            shutil.copy2(src, dst)
            self.df = self.df.append({
                'original_filename': orig,
                'new_filename'     : new,
                'label'            : lbl,
                'timestamp'        : timestamp
            }, ignore_index=True)
            self.df.to_csv(self.csv_path, index=False)
            logging.info(f"✅ {orig} → {new}")
            self.status.value = f"Labeled: {new}"
        except Exception as e:
            logging.error(f"Error: {e}")
            self.status.value = f"Error: {e}"

        self.i += 1
        if self.i < len(self.images):
            self._render()
        else:
            clear_output(wait=True)
            print("🎉 All done! Metadata in labels.csv")

    def _render(self):
        fname = self.images[self.i]
        self._show_image(os.path.join(self.src, fname))
        self.status.value = f"Image {self.i+1}/{len(self.images)}: {fname}"

    def start(self):
        if not self.images:
            print("No unlabeled images found.")
            return
        self._render()
        ui = widgets.VBox([self.out,
                           widgets.HBox([self.dropdown, self.button]),
                           self.status])
        display(ui)


In [ ]:
# 1.4 Launch Labeler
if __name__ == '__main__':
    labeler = ColabImageLabeler(
        src_dir=BASE_DIR,
        dst_dir=LABELED_DIR,
        csv_path=CSV_PATH,
        labels=LABELS
    )
    labeler.start()


### 2. Import Required Libraries for the Rest of the Project

In [ ]:
# 1. Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 3. Dataset Creation

In [ ]:
# TODO: code for splitting data, creating train/val folders, etc. …

### 4. Model Training

In [ ]:
# TODO: code for defining and training your CNN …

### 5. Evaluation & Deployment

In [ ]:
# TODO: code for evaluating accuracy, exporting a TensorFlow Lite model, etc. …